In [25]:
#数据准备与处理
#？？？为什么是灰度图像？

In [2]:
import os
from PIL import Image

In [82]:
#制作简易mask（未消除柱子）——二值化处理
image_root='E:/工作文件/机器学习/HF片子/dataset'
images=os.listdir(os.path.join(image_root,"transfer"))#注意这里会乱序，如果需要考虑名称和顺序的要改
for i in range(len(images)):
    img=images[i]
    img_file=os.path.join(image_root,"transfer",img)
    image=Image.open(img_file)
    image.save(os.path.join(image_root,"train",str(i+1+222)+'.jpg'))
    image=image.convert("L")
    threshold=50
    table=[]
    for j in range(256):
        if j < threshold:
            table.append(0)
        else:
            table.append(1)
    black=image.point(table,'1')#一个通过列表table的映射，‘1’表示mode
    """
    PIL.Image中的mode
    '1'：位图，像素1位
    'L'：灰度，像素8位
    'I'：像素int32
    'F'：像素float32
    'P'：8位，映射为其他模式
    'RGB'：真色彩，3通道
    'RGBA'：4通道，加透明
    'CMYK'：印刷，4通道
    'YCbCr'：亮色分离，3通道
    """
    black.save(os.path.join(image_root,"train_mask",str(i+1+222)+'.jpg'))

In [83]:
#保存成灰度图像
if __name__ == '__main__':
    #原始数据目录
    data_root='E:/工作文件/机器学习/HF片子/dataset'
    #train目录下图片文件名
    train_names=os.listdir(os.path.join(data_root,"train"))
    mask_names=os.listdir(os.path.join(data_root,"train_mask"))
    
    #处理train目录下的文件
    for train_name in train_names:
        #得到图片文件路径
        image_file=os.path.join(data_root,"train",train_name)
        #读取图像并convert成灰度图，“L”表示灰度
        image=Image.open(image_file).convert("L")
        #不存在的话创建文件夹
        if not os.path.exists(os.path.join(data_root,"Images","train")):
            os.makedirs(os.path.join(data_root,"Images","train"))
        #保存图片
        image.save(os.path.join(data_root,"Images","train",train_name))
    
    #处理train_mask目录下的文件
    for mask_name in mask_names:
        #得到图片文件路径
        image_file=os.path.join(data_root,"train_mask",mask_name)
        #读取图像并convert成灰度图，“L”表示灰度
        image=Image.open(image_file).convert("L")
        #不存在的话创建文件夹
        if not os.path.exists(os.path.join(data_root,"Images","train_mask")):
            os.makedirs(os.path.join(data_root,"Images","train_mask"))
        #保存图片
        image.save(os.path.join(data_root,"Images","train_mask",mask_name))

In [84]:
#制作TFrecord
import tensorflow as tf

if __name__=='__main__':
    #灰度图像目录
    data_root='E:/工作文件/机器学习/HF片子/dataset/Images'
    #Images/train目录下图像文件保存到列表
    image_names = os.listdir(os.path.join(data_root, "train")) 
    label_names = os.listdir(os.path.join(data_root, "train_mask")) 
    
    #创建tfrecord目录
    if not os.path.exists(os.path.join("E:/工作文件/机器学习/HF片子/dataset", "tfrecords")):
        os.makedirs(os.path.join("E:/工作文件/机器学习/HF片子/dataset", "tfrecords"))
    #第一步：创建文件dataset\tfrecords\image.tfrecords，为该文件夹创建TFRecordWriter准备写入数据
    writer=tf.python_io.TFRecordWriter(os.path.join("E:/工作文件/机器学习/HF片子/dataset", "tfrecords","image.tfrecords"))
    feature = {'label': None, 'data': None}
    for i in range(0,len(image_names)):
        #得到训练原图像路径
        image_raw_file=os.path.join(data_root,"train",image_names[i])
        #得到训练图像mask
        image_label_file=os.path.join(data_root,"train_mask",label_names[i])
        
        #第二步：读取未解码的原图及mask
        # tf.gfile.FastGFile('path',mode).read()函数：读取没有经过解码的原始图像，如果要显示读入的图像，那就需要经过解码过程，读取的图像是一个字符串，没法显示示
        image_raw = tf.gfile.FastGFile(image_raw_file, 'rb').read() 
        image_label = tf.gfile.FastGFile(image_label_file, 'rb').read()
        """
        tfrecord文件包含了tf.train.Example 协议缓冲区(protocol buffer，协议缓冲区包含了特征 Features)。
        你可以写一段代码获取你的数据， 将数据填入到Example协议缓冲区(protocol buffer)，将协议缓冲区序列化为一个字符串，
        并且通过tf.python_io.TFRecordWriter class写入到TFRecords文件。
        
        缓冲区格式如下：
        tf.train.Example(features=tf.train.Features(feature={key:value,key:value,...})
        tf.train.Features是tf.train.Feature的复数，事实上tf.train.Features有属性为feature，
        这个属性的一般设置方法是传入一个字典，字典的key是字符串（feature名），而value是tf.train.Feature对象
        
        tf.train.Feature有三个属性,支持整型、浮点数和二进制三种格式，分别是
        tf.train.Feature(int64_list=tf.train.Int64List(value=[int_scalar]))
        tf.train.Feature(bytes_list=tf.train.BytesList(value=[array_string_or_byte]))
        tf.train.Feature(float_list=tf.train.FloatList(value=[float_scalar]))
        write bytes to Example proto buffer
        """
        #第三步：将raw及label填入到tfrecord文件Example缓冲区
        #图像整合成二进制数据
        image_list=tf.train.BytesList(value=[image_raw])
        label_list=tf.train.BytesList(value=[image_label])
        #使用tf.train.feature【注意没有s】将图像作为value输入到字典中
        feature['data'] = tf.train.Feature(bytes_list=image_list)
        feature['label'] = tf.train.Feature(bytes_list=label_list)
        #使用tf.train.features【注意有s】将图像和标签的字典输入到Example缓冲区
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        
        
        #第四步：Example缓冲区序列化并写入到tfrecord文件
        writer.write(example.SerializeToString())
    writer.close

D:\software\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\software\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\software\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\software\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or 